<a href="https://colab.research.google.com/github/mikeobeid/CloudCourse/blob/main/ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial 4 - Cloud Computing**
**Project:** Vehicle Data Visualization (Sunburst + Heatmap + Tabs)

**Data Source:** [https://data.gov.il](https://data.gov.il)

**Summary:**  
This notebook loads vehicle registration data from a public API and provides a combined interactive visualization using a Sunburst chart and a Heatmap.  
It also includes a tabbed dashboard for exploring summary statistics, raw data, yearly production distribution, and model-specific trims.



In [ ]:
# ========== Import Libraries ==========
import pandas as pd
import requests
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go


In [ ]:
# ========== Install Libraries ==========
!pip install plotly
!pip install seaborn

In [ ]:
# ========== Load Data ==========
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3&limit=1000'
response = requests.get(url)
data = response.json()
data_df = pd.DataFrame(data['result']['records'])
data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')


In [ ]:
# ========== Create Sunburst Chart ==========
filtered_sunburst_df = data_df[
    data_df['tozeret_nm'].notna() &
    data_df['kinuy_mishari'].notna() &
    data_df['ramat_gimur'].notna()
].copy()

filtered_sunburst_df['id'] = (
    filtered_sunburst_df['tozeret_nm'] + " - " +
    filtered_sunburst_df['kinuy_mishari'] + " - " +
    filtered_sunburst_df['ramat_gimur']
)

filtered_sunburst_df['parent'] = (
    filtered_sunburst_df['tozeret_nm'] + " - " +
    filtered_sunburst_df['kinuy_mishari']
)

level1 = filtered_sunburst_df[['tozeret_nm']].drop_duplicates().rename(columns={'tozeret_nm': 'label'})
level1['parent'] = ""
level1['id'] = level1['label']

level2 = filtered_sunburst_df[['tozeret_nm', 'kinuy_mishari']].drop_duplicates()
level2['label'] = level2['kinuy_mishari']
level2['parent'] = level2['tozeret_nm']
level2['id'] = level2['tozeret_nm'] + " - " + level2['kinuy_mishari']

level3 = filtered_sunburst_df[['id', 'ramat_gimur', 'parent']].rename(columns={'ramat_gimur': 'label'})

sunburst_df = pd.concat([
    level1[['id', 'label', 'parent']],
    level2[['id', 'label', 'parent']],
    level3[['id', 'label', 'parent']]
], ignore_index=True)


In [ ]:
# ========== Prepare Heatmap ==========
heatmap_df = data_df.dropna(subset=['tozeret_nm', 'shnat_yitzur'])
heatmap_pivot = heatmap_df.pivot_table(index='tozeret_nm', columns='shnat_yitzur', aggfunc='size', fill_value=0)

z = heatmap_pivot.values
x = heatmap_pivot.columns.astype(str)
y = heatmap_pivot.index


In [ ]:
# ========== Combine Sunburst and Heatmap ==========
combined_fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'domain'}, {'type': 'heatmap'}]],
    column_widths=[0.45, 0.55],
    subplot_titles=["Sunburst🌞: Click To Expand", "Heatmap 📊 : Makes vs. Years"]
)

# Add Sunburst
combined_fig.add_trace(
    go.Sunburst(
        ids=sunburst_df['id'],
        labels=sunburst_df['label'],
        parents=sunburst_df['parent'],
        maxdepth=3
    ),
    row=1, col=1
)

# Add Heatmap
combined_fig.add_trace(
    go.Heatmap(z=z, x=x, y=y, colorscale='YlGnBu'),
    row=1, col=2
)

combined_fig.update_layout(
    title_text="Combined Visualization: Vehicle Distribution",
    height=650,
    margin=dict(t=50, l=0, r=0, b=0)
)

combined_fig.show()


In [ ]:
# ========== Tab 1: Overview ==========
tab1 = widgets.Output()
with tab1:
    display(data_df.describe())
    plt.figure(figsize=(10, 4))
    sns.histplot(data_df['shnat_yitzur'].dropna(), bins=20, kde=True, color='royalblue')
    plt.title("Distribution of Production Years")
    plt.xlabel("Production Year")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# ========== Tab 2: Raw Data ==========
tab2 = widgets.Output()
with tab2:
    display(data_df.head(15))


In [ ]:
# ========== Tab 3: Year Chart ==========
tab3 = widgets.Output()
with tab3:
    year_counts = data_df['shnat_yitzur'].value_counts().sort_index()
    plt.figure(figsize=(12, 5))
    plt.bar(year_counts.index.astype(int), year_counts.values, color='mediumseagreen')
    plt.xlabel("Production Year")
    plt.ylabel("Number of Vehicles")
    plt.title("Number of Vehicles by Year")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# ========== Tab 4: Trim Explorer ==========
tab4 = widgets.Output()

tozeret_nm_dropdown = widgets.Dropdown(
    options=[''] + sorted(data_df['tozeret_nm'].dropna().unique()),
    description='Tozeret:',
    style={'description_width': 'initial'}
)

kinuy_mishari_dropdown = widgets.Dropdown(
    options=[''],
    description='Kinuy Mishari:',
    style={'description_width': 'initial'}
)

output_area = widgets.Output()

def update_kinuy_mishari_options(change):
    if change['new']:
        models = data_df[data_df['tozeret_nm'] == change['new']]['kinuy_mishari'].dropna().unique()
        kinuy_mishari_dropdown.options = [''] + sorted(models)
    else:
        kinuy_mishari_dropdown.options = ['']

def update_output(change=None):
    output_area.clear_output()
    selected_tozeret = tozeret_nm_dropdown.value
    selected_kinuy = kinuy_mishari_dropdown.value

    if selected_tozeret and selected_kinuy:
        filtered_df = data_df[
            (data_df['tozeret_nm'] == selected_tozeret) &
            (data_df['kinuy_mishari'] == selected_kinuy)
        ]
        total_records = len(filtered_df)
        unique_ramat_gimur = filtered_df['ramat_gimur'].dropna().unique()

        with output_area:
            print(f"Total Records: {total_records}")
            print(f"Unique Ramat Gimur: {', '.join(unique_ramat_gimur) if len(unique_ramat_gimur) > 0 else 'None'}")
    else:
        with output_area:
            print("Please select valid options for both dropdowns.")

tozeret_nm_dropdown.observe(update_kinuy_mishari_options, names='value')
kinuy_mishari_dropdown.observe(update_output, names='value')

with tab4:
    display(widgets.VBox([tozeret_nm_dropdown, kinuy_mishari_dropdown, output_area]))


In [ ]:
# ========== Display all Tabs ==========
tabs = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tabs.set_title(0, "Overview")
tabs.set_title(1, "Raw Data")
tabs.set_title(2, "Year Chart")
tabs.set_title(3, "Trim Explorer")
display(tabs)
